##### Idea
The idea of this simple project is to get familiar with multivariate linear regression as a continuation of linear regression.
A regressor has to be made from scratch that we will use to make a model consisting of two variables.

**A little bit about the dataset:**  
A psychiatrist assigns values for mental retardation and degree of distrust of doctors in 53 newly hospitalized patients.  
Six months later, a value for the degree of illness of each patient is assigned.  
The question is whether the effect of treatment can be related to these indices.  
Since the independent and dependent variables are essentially subjective, it is assumed that large errors are made in all of them.  

There are 53 rows of data.  
The data comprises:  
- The index;
- The degree of mental retardation;
- The degree of distrust of doctors;
- The degree of illness.

We seek a model of the form:
$B = \theta{}_1 * X_1 + \theta{}_2 * X_2$



In [1]:
# import jtplot submodule from jupyterthemes
from jupyterthemes import jtplot

# currently installed theme will be used to
# set plot style if no arguments provided
jtplot.style()

In [2]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
class MultiVariateLinearRegression():
    
    # Define the parameters required
    epochs = 0
    lr = 0.0
    variate_count = 0
    theta = None
    optimizer = None
    cost_function = None
    output_result = False
    
    """
        Initiate the model

        Keyword arguments:
        variate_count -- The amount of features the model is supposed to have
        lr -- The learning rate used during training
        epochs -- The amount of epochs to train for
    """
    def __init__(self, variate_count, lr = 0.01, epochs = 500, output_result = False):
        self.lr = lr
        self.epochs = epochs
        self.variate_count = variate_count
        self.output_result = output_result
        # Creates a 1 x M vector of zeros as a starting point
        self.theta = np.zeros((1, variate_count))
                
    """
        Add the cost function that can measure the loss

        Keyword arguments:
        cost_function -- A pointer to the function to be used during gradient descent
    """
    def cost_function(self, cost_function):
        
        # The cost_function should actually be a function
        assert callable(cost_function)
        
        self.cost_function = cost_function
        
    """
        Add the optimizer function that takes care of the gradient descent step

        Keyword arguments:
        optimizer -- A pointer to the function to be used during gradient descent
    """
    def optimizer(self, optimizer):
        
        # The cost_function should actually be a function
        assert callable(optimizer)
        
        self.optimizer = optimizer
    
    """
        Fits the model to the given data

        Keyword arguments:
        X -- The feature set
        y -- The labels
    """
    def fit(self, X, y):
        
        # See if the cost function is set
        assert callable(self.optimizer)
        
        # Check if the amount of variates is correct
        assert self.variate_count == X.shape[1]
        
        # Train for the amount of epochs
        for e in range(self.epochs):
            # Update theta
            self.theta = self.theta - (self.lr * self.optimizer(X, y, self.theta))
            
            if callable(self.cost_function) and self.output_result and e % int(epochs / 10) == 0:
                print("Epoch {} Cost: {}".format(e+1, self.cost_function(X, y, self.theta)))
            
        if callable(self.cost_function) and self.output_result:
            print("Final Cost: {}".format(self.cost_function(X, y, self.theta)))
            
        return self.theta.flatten()
    
    """
        Predicts values based on given dataset

        Keyword arguments:
        X -- The feature set
    """
    def predict(self, X):
        return X.dot(self.theta.T)

In [4]:
"""
    Computes the Mean Squared Error cost function

    Keyword arguments:
    X -- The feature set
    y -- The labels
"""
def MSE ( X, y, theta ):
    # Find the amount of samples
    m = len(X)
    # Calculate the 'cost'
    cost = np.dot(X, theta.T) - y
    # Square the summand
    cost = np.power(cost, 2)
    # Calculate the mean
    return np.sum(cost) / (2*m)

"""
    Computes the Mean Squared Error derivative function

    Keyword arguments:
    X -- The feature set
    y -- The labels
    theta -- The variates
"""
def MSEDeriv ( X, y, theta ):    
    return (1/len(X)) * np.sum(X * (X @ theta.T - y), axis=0)

In [5]:
###
### Data preparation
###

# Load in the data
dataset = pd.read_csv("data/retardation/dataset.txt", sep = '  ', 
                      names = ["id", "Retardation Index", "Distrust Index", "Degree of Illness"], engine='python')

# Since pandas provides excellent indexing, we will drop the id column
dataset = dataset.drop("id", axis = 1)

# Print the first five rows
dataset.head(5)

,Retardation Index,Distrust Index,Degree of Illness
0,2.80,6.1,44
1,3.10,5.1,25
2,2.59,6.0,10
3,3.36,6.9,28
4,2.80,7.0,25


In [6]:
# We will now separate the independent variables from the dependent variable
# and convert them to numpy
dataset = np.asarray(dataset)

# 0: Retardation index
# 1: Distrust index
# 2: Degree of illness
X, y = dataset[:,[1,2]], dataset[:,0].reshape(-1, 1)

In [7]:
###
### Training
###
amount_of_dependent_variables = 2
learning_rate = 0.0005
epochs = 1000

# Define our multi variate model
model = MultiVariateLinearRegression(amount_of_dependent_variables, 
                                     lr = learning_rate, epochs = epochs, 
                                     output_result = True)

# Define the appropriate cost function
model.cost_function(MSE)
# Define the optimizer
model.optimizer(MSEDeriv)
# Train the model on the dataset
parameters = model.fit(X, y)

Epoch 1 Cost: 2.427770270852128
Epoch 101 Cost: 0.31143469996570283
Epoch 201 Cost: 0.10394340948365398
Epoch 301 Cost: 0.05923015256563651
Epoch 401 Cost: 0.049594686230775405
Epoch 501 Cost: 0.04751829506874331
Epoch 601 Cost: 0.047070843943086235
Epoch 701 Cost: 0.04697442063253048
Epoch 801 Cost: 0.046953641927806336
Epoch 901 Cost: 0.046949164228715416
Final Cost: 0.046948203408104336


In [8]:
# Our final model is:
print("Final model: {} x1 + {} x2".format(parameters[0], parameters[1]))

Final mode: 0.4132848623364123 x1 + 0.01048734904946764 x2
